# Data Cleaning

In this notebook, we clean the data, so that it can be used for model training.

In [54]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/reddit_dataset.csv')
df.head()

,subreddit,id,title,selftext,score,num_comments,created_utc,flair,upvote_ratio,is_self,nsfw,author,url,sort_type,popularity_bucket
0,technology,1ly9hhz,Generative AI is Turning Publishing Into a Swa...,NaN,0,0,1.752350e+09,Artificial Intelligence,0.50,False,False,ubcstaffer123,https://www.pastemagazine.com/books/publishing...,new,low
1,technology,1ly9dpp,Can AI help prevent homelessness?,NaN,0,5,1.752350e+09,Society,0.40,False,False,ubcstaffer123,https://www.thedesertreview.com/news/state/can...,new,low
2,technology,1ly851k,Why forecasters are concerned about losing 3 k...,NaN,30,6,1.752347e+09,Society,0.94,False,False,Wagamaga,https://www.pbs.org/newshour/science/why-forec...,new,low
3,technology,1ly7o4a,Is AI Taking Over Influencers' Jobs?,NaN,0,22,1.752345e+09,Artificial Intelligence,0.33,False,False,ThickTrack1492,https://www.aiviralclub.com/is-ai-taking-over-...,new,low
4,technology,1ly6fe5,The Trump Administration Is Planning to Use AI...,NaN,3369,237,1.752342e+09,Artificial Intelligence,0.98,False,False,Aggravating_Money992,https://truthout.org/articles/the-trump-admini...,new,medium


### Data inspection

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10047 entries, 0 to 10046
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   subreddit          10047 non-null  object 
 1   id                 10047 non-null  object 
 2   title              10047 non-null  object 
 3   selftext           1335 non-null   object 
 4   score              10047 non-null  int64  
 5   num_comments       10047 non-null  int64  
 6   created_utc        10047 non-null  float64
 7   flair              6008 non-null   object 
 8   upvote_ratio       10047 non-null  float64
 9   is_self            10047 non-null  bool   
 10  nsfw               10047 non-null  bool   
 11  author             9641 non-null   object 
 12  url                10047 non-null  object 
 13  sort_type          10047 non-null  object 
 14  popularity_bucket  10047 non-null  object 
dtypes: bool(2), float64(2), int64(2), object(9)
memory usage: 1.0+ MB


We see here that most of the columns (or variables) contain no missing values, except for `selftext`, `flair` and `author`.

In [5]:
df.describe()

,score,num_comments,created_utc,upvote_ratio
count,10047.000000,10047.000000,1.004700e+04,10047.000000
mean,42127.680402,1548.207823,1.665287e+09,0.821978
std,45791.415401,2682.173433,9.222974e+07,0.184150
min,0.000000,0.000000,1.407798e+09,0.060000
25%,127.000000,31.000000,1.588867e+09,0.810000
50%,38868.000000,824.000000,1.682469e+09,0.890000
75%,72902.500000,2252.500000,1.751457e+09,0.930000
max,439029.000000,88172.000000,1.752052e+09,1.000000


### 1. Dealing with missing values
#### i. `selftext`

Since the `selftext` column represents the text (or the body) of a "text-only" Reddit post, we check to see if it's the same as the title of the post:

In [ ]:
# analyis of the selftext column: Is it just the same as title field?
# Show only rows where selftext is not missing
df_with_selftext = df[df['selftext'].notna() & (df['selftext'].str.strip() != '')]

# View first few rows
df_with_selftext[['title', 'selftext']].head(10)

,title,selftext
813,Impromptu Guinness Chugging Contest at the Nat...,Badlands wins by a sip at the Gaf West in Hell...
859,Wimbledon Round 1: Arthur Rinderknech has knoc...,Arthur Rinderknech has knocked out third seed ...
872,The WNBA is expanding to 18 teams over the nex...,"Cleveland will begin play in 2028, Detroit in ..."
877,Crazy flip in Argentina's TC Pick Up,Driver Agustín Martínez is concious and was ta...
883,15yr old Australian skateboarder Arisa Trew br...,"At just 15 years old, Arisa Trew has once agai..."
892,Kishane Thompson of Jamaica has run the sixth-...,Thompson is firmly in the mix for gold at the ...
894,“Best day of my life” - Dunne delivers on F1 d...,https://www.bbc.com/sport/formula1/articles/cj...
914,15 years ago today in 2010 - Uruguay beat Sout...,La Celeste would advance to the World Cup semi...
930,New York Mets Pitcher Richard Lovelady Request...,What's in a name?
941,"For The First Time In History, the Reigning NB...",Canadian athletes are having a good year going...


It's not the same as the title column, hence we cannot discard it.

Since a NaN value in selftext means that the Reddit post contains no body text (it’s just a link post or a title-only post), we go ahead and fill these cells with blank (to show no body text).

In [ ]:
df['selftext'] = df['selftext'].fillna('')   # Keep blank text if missing

#### ii. `author`

On Reddit when the post and comment information is available, but not the author information, it implies that the user account was deleted (or that Reddit moderators removed the post or user). 

Hence we replace the NaN values with the value `[deleted]`.


In [ ]:
df['author'] = df['author'].fillna('[deleted]')

#### iii. `flair`

Since "flair" is an optional tag on Reddit, not every post has a flair. This is what caused the NaN values, therefore we set the `flair` tag for these posts to `None`.

In [ ]:
df['flair'] = df['flair'].fillna('None')     # Fill missing flair with "none" label (as no label assigned)

df.drop_duplicates(subset='id', inplace=True)

We extract meaningful information from the timstamp field (`created_utc`), by retaining the hour at which the post was created, and then drop the field (as it's now redundant).

In [ ]:
# Create a new feature (hour) from timestamp
df['created_hour'] = pd.to_datetime(df['created_utc'], unit='s').dt.hour
df.drop(columns=['created_utc'], inplace=True)

In [12]:
df.head()

,subreddit,id,title,selftext,score,num_comments,flair,upvote_ratio,is_self,nsfw,author,url,sort_type,popularity_bucket,created_hour
0,technology,1lvds7w,Students can’t use AI to cheat on standardized...,,2,2,Artificial Intelligence,1.00,False,False,ubcstaffer123,https://www.fraserinstitute.org/commentary/stu...,new,low,8
1,technology,1lvdi5e,Instagram wrongly accuses some users of breach...,,9,2,Social Media,1.00,False,False,zsreport,https://www.bbc.com/news/articles/cy8kjdz9nr3o,new,low,8
2,technology,1lvcxoa,Turkey blocks X's Grok chatbot for alleged ins...,,18,3,Social Media,0.91,False,False,BreakfastTop6899,https://www.reuters.com/business/media-telecom...,new,low,7
3,technology,1lvai0d,GlobalFoundries to make RISC-V CPUs — fab acqu...,,18,1,Hardware,0.83,False,False,jhansonxi,https://www.tomshardware.com/pc-components/cpu...,new,low,5
4,technology,1lv9syt,Rubio impersonation campaign underscores broad...,,25,3,Artificial Intelligence,0.82,False,False,BreakfastTop6899,https://www.axios.com/2025/07/08/rubio-ai-impe...,new,low,4


We confirm that there are no more null values:

In [13]:
df.isna().sum()

subreddit            0
id                   0
title                0
selftext             0
score                0
num_comments         0
flair                0
upvote_ratio         0
is_self              0
nsfw                 0
author               0
url                  0
sort_type            0
popularity_bucket    0
created_hour         0
dtype: int64

In [14]:
print(df['flair'].nunique())
print(df['author'].nunique())
print(df['subreddit'].nunique())

189
5032
7


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10047 entries, 0 to 10046
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   subreddit          10047 non-null  object 
 1   id                 10047 non-null  object 
 2   title              10047 non-null  object 
 3   selftext           10047 non-null  object 
 4   score              10047 non-null  int64  
 5   num_comments       10047 non-null  int64  
 6   flair              10047 non-null  object 
 7   upvote_ratio       10047 non-null  float64
 8   is_self            10047 non-null  bool   
 9   nsfw               10047 non-null  bool   
 10  author             10047 non-null  object 
 11  url                10047 non-null  object 
 12  sort_type          10047 non-null  object 
 13  popularity_bucket  10047 non-null  object 
 14  created_hour       10047 non-null  int32  
dtypes: bool(2), float64(1), int32(1), int64(2), object(9)
memory usage: 10

In [16]:
df.nunique()

subreddit                7
id                   10047
title                 9985
selftext              1326
score                 6822
num_comments          3703
flair                  189
upvote_ratio            95
is_self                  2
nsfw                     2
author                5032
url                  10014
sort_type                2
popularity_bucket        3
created_hour            24
dtype: int64

#### The `url` field

The `url` field has a very high cardinality (almost unique for each post), and with further inspection, it seems that we could group the urls by domain:

In [ ]:
# Extracting domains from url

from urllib.parse import urlparse

df['url_domain'] = df['url'].apply(lambda x: urlparse(x).netloc.lower())
domain_counts = df['url_domain'].value_counts()

# Is it worth grouping by domains alone? (Ans: No)
# For example, count how many domains appear less than or equal to 5 times:
rare_domains = domain_counts[domain_counts <= 5]

print(f"Total unique domains: {df['url_domain'].nunique()}")    
print(f"Number of domains with 5 or fewer posts: {len(rare_domains)}")
print(rare_domains)

Total unique domains: 1193
Number of domains with 5 or fewer posts: 1023
url_domain
www.miamiherald.com                    5
www.researchgate.net                   5
www.rockpapershotgun.com               5
www.congress.gov                       5
www.livescience.com                    5
                                      ..
www.manchester.ac.uk                   1
www.annfammed.org                      1
momentummag.com                        1
bvajournals.onlinelibrary.wiley.com    1
www.oddfilms.com                       1
Name: count, Length: 1023, dtype: int64


But we see here, that out from the total number of domains, most (1023 out of 1193 unique domains) have very few posts. 

In [61]:
print(df['url_domain'].value_counts().head(10))  # top 10 most common domains

url_domain
i.redd.it                  1600
www.reddit.com             1082
v.redd.it                   754
i.imgur.com                 498
www.independent.co.uk       164
www.businessinsider.com     159
gfycat.com                  159
www.espn.com                157
www.psypost.org             136
www.newsweek.com            109
Name: count, dtype: int64


From here, we note that Reddit-hosted domains (like i.redd.it, v.redd.it, and www.reddit.com) are most dominant. On further analysis of these domains, we find that:

i)   i.redd.it - is Reddit's image hosting domain

ii)  v.redd.it - is Reddit's video hosting domain and

iii) www.reddit.com, old.reddit.com and np.reddit.com - links to other Reddit posts or comments

Since most URLs fall into these categories, we create a new field, `media_type`, to categorize posts as image, video, internal Reddit link, or external link based on their URLs.

In [ ]:
# Extracting media information (such as if urls contain images, videos, or points to external links):

# Define internal Reddit domains related to media and posts
internal_domains = ['reddit.com', 'old.reddit.com', 'np.reddit.com']
image_domains = ['i.redd.it']
video_domains = ['v.redd.it']

def classify_media(domain, is_self):
    if is_self:
        return 'self_post'  # text/self post
    elif domain in image_domains:
        return 'image'
    elif domain in video_domains:
        return 'video'
    elif domain in internal_domains:
        return 'internal_link'
    else:
        return 'external_link'

df['media_type'] = df.apply(lambda row: classify_media(row['url_domain'], row['is_self']), axis=1)

This grouping seems more meaningful:

In [ ]:
# A more meaningful grouping
print(df['media_type'].value_counts())

media_type
external_link    6665
image            1584
self_post        1070
video             727
internal_link       1
Name: count, dtype: int64


In [31]:
df.head()

,subreddit,id,title,selftext,score,num_comments,flair,upvote_ratio,is_self,nsfw,author,url,sort_type,popularity_bucket,created_hour,url_domain,media_type
0,technology,1lvds7w,Students can’t use AI to cheat on standardized...,,2,2,Artificial Intelligence,1.00,False,False,ubcstaffer123,https://www.fraserinstitute.org/commentary/stu...,new,low,8,www.fraserinstitute.org,external_link
1,technology,1lvdi5e,Instagram wrongly accuses some users of breach...,,9,2,Social Media,1.00,False,False,zsreport,https://www.bbc.com/news/articles/cy8kjdz9nr3o,new,low,8,www.bbc.com,external_link
2,technology,1lvcxoa,Turkey blocks X's Grok chatbot for alleged ins...,,18,3,Social Media,0.91,False,False,BreakfastTop6899,https://www.reuters.com/business/media-telecom...,new,low,7,www.reuters.com,external_link
3,technology,1lvai0d,GlobalFoundries to make RISC-V CPUs — fab acqu...,,18,1,Hardware,0.83,False,False,jhansonxi,https://www.tomshardware.com/pc-components/cpu...,new,low,5,www.tomshardware.com,external_link
4,technology,1lv9syt,Rubio impersonation campaign underscores broad...,,25,3,Artificial Intelligence,0.82,False,False,BreakfastTop6899,https://www.axios.com/2025/07/08/rubio-ai-impe...,new,low,4,www.axios.com,external_link


In [ ]:
# We now drop the columns url and url_domain:

df.drop(columns=['url', 'url_domain'], inplace=True)
df.head()

,subreddit,id,title,selftext,score,num_comments,flair,upvote_ratio,is_self,nsfw,author,sort_type,popularity_bucket,created_hour,media_type
0,technology,1lvds7w,Students can’t use AI to cheat on standardized...,,2,2,Artificial Intelligence,1.00,False,False,ubcstaffer123,new,low,8,external_link
1,technology,1lvdi5e,Instagram wrongly accuses some users of breach...,,9,2,Social Media,1.00,False,False,zsreport,new,low,8,external_link
2,technology,1lvcxoa,Turkey blocks X's Grok chatbot for alleged ins...,,18,3,Social Media,0.91,False,False,BreakfastTop6899,new,low,7,external_link
3,technology,1lvai0d,GlobalFoundries to make RISC-V CPUs — fab acqu...,,18,1,Hardware,0.83,False,False,jhansonxi,new,low,5,external_link
4,technology,1lv9syt,Rubio impersonation campaign underscores broad...,,25,3,Artificial Intelligence,0.82,False,False,BreakfastTop6899,new,low,4,external_link


### Text cleaning

We now perform basic cleaning on the text columns `title` and `self_text`, by lowercasing, removing HTML tags, URLs and extra white spaces:

In [48]:
print(df['selftext'].str.contains(r'<.*?>').sum())
print(df['title'].str.contains(r'<.*?>').sum())

3
0


In [49]:
print(df['selftext'].str.contains(r'http\S+|www\.\S+').sum())
print(df['title'].str.contains(r'http\S+|www\.\S+').sum())

160
0


In [ ]:
import re

# A basic cleaning function
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+|www.\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Appling to title and selftext:
df['title'] = df['title'].apply(clean_text)
df['selftext'] = df['selftext'].apply(clean_text)

In [51]:
df.head()

,subreddit,id,title,selftext,score,num_comments,flair,upvote_ratio,is_self,nsfw,author,sort_type,popularity_bucket,created_hour,media_type
0,technology,1lvds7w,students can’t use ai to cheat on standardized...,,2,2,Artificial Intelligence,1.00,False,False,ubcstaffer123,new,low,8,external_link
1,technology,1lvdi5e,instagram wrongly accuses some users of breach...,,9,2,Social Media,1.00,False,False,zsreport,new,low,8,external_link
2,technology,1lvcxoa,turkey blocks x's grok chatbot for alleged ins...,,18,3,Social Media,0.91,False,False,BreakfastTop6899,new,low,7,external_link
3,technology,1lvai0d,globalfoundries to make risc-v cpus — fab acqu...,,18,1,Hardware,0.83,False,False,jhansonxi,new,low,5,external_link
4,technology,1lv9syt,rubio impersonation campaign underscores broad...,,25,3,Artificial Intelligence,0.82,False,False,BreakfastTop6899,new,low,4,external_link


In [53]:
len(df)

10047

We finally save the cleaned data to "cleaned_reddit_posts.csv".

In [ ]:
df.to_csv('../data/cleaned_reddit_posts.csv', index=False)